<a href="https://colab.research.google.com/github/aroonalok/zoho-apis/blob/changes/ZohoBooksReports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [332]:
from bs4 import BeautifulSoup
from datetime import datetime
from google.colab import files
import IPython
import openpyxl
from openpyxl.styles import Font, Color, Border, Side
import pandas as pd
import pytz
import re
import requests

# Configuration

*   Go to https://api-console.zoho.in/
*   Click on the `Self Client`
*   Copy `Client ID` and `Client Secret` from `Client Secret` tab.
*   Organization ID can be left unconfigured
*   Auth Token can be left unconfigured if you have the downloaded Token file (`authorization_token.txt`) in your local machine.

In [333]:
# @title Access Config

ORGANIZATION_ID = 0 # @param {type:"integer"}
AUTH_TOKEN = "" # @param {type:"string"}
CLIENT_ID = "" # @param {type:"string"}
CLIENT_SECRET = "" # @param {type:"string"}

if not (CLIENT_ID and CLIENT_SECRET):
  raise AssertionError("Fill up Client ID and Client Secret values!")

In [334]:
# @title Date Config
FROM_DATE = '2024-04-02' # @param {type:"date"}
TO_DATE = '2024-04-03' # @param {type:"date"}
DOWNLOAD_EXCEL_REPORT = True # @param {type:"boolean"}
DOWNLOAD_HTML_REPORT = True # @param {type:"boolean"}

DATE_FORMAT = '%Y-%m-%d'
if datetime.strptime(FROM_DATE, DATE_FORMAT) > datetime.strptime(TO_DATE, DATE_FORMAT):
  raise AssertionError("TO_DATE should be greater than FROM_DATE ")

In [335]:
# @title API Config

MAX_RECORDS_PER_PAGE = 1000
IST_TIMEZONE = pytz.timezone('Asia/Kolkata')

API_CALLS_COUNTER = 0

ROOT_API_ENDPOINT = "https://www.zohoapis.in/books/v3/"
OAUTH_ROOT_URL = "https://accounts.zoho.in/oauth/v2/token?"
OAUTH_REDIRECT_URI = "http://www.zoho.in/books"

In [336]:
# @title Report Config

PAYMENT_MODE_CREDIT = "Credit"
FROM_TO_DATE = "FROM: {from_date} TO: {to_date}"
FOR_DATE = "FOR: {for_date}"
DAILY_SALES_REPORT_CAPTION = "DAILY SALES SUMMARY REPORT {time_range}"
SALES_TYPE_REPORT_CAPTION = "SALES TYPE SUMMARY REPORT {time_range}"
INTRA_STATE_TAX_REPORT_CAPTION = "INTRA STATE TAX REPORT {time_range}"
TOTAL_INTRA_STATE_TAX_REPORT_CAPTION = "INTRA STATE TAX SUMMARY REPORT {time_range}"
FILENAME = "report-from-{from_date}-to-{to_date}.{extension}"
SALES_SHEET_NAME = "Sales"
SALES_TYPE_SHEET_NAME = "Sales Type"
GST_SHEET_NAME = "GST"
TOTAL_GST_SHEET_NAME = "TOTAL GST"
NO_DATA_MESSAGE = "NO DATA PRESENT {time_range}"

REPORT_TAB_TITLE = "" # @param {type:"string"}
LOGO_URL = "" # @param {type:"string"}
ORGANIZATION_NAME = "" # @param {type:"string"}
ADDRESS_LINE1 =  "" # @param {type:"string"}
ADDRESS_LINE2 = '' # @param {type:"string"}
PHONE = '' # @param {type:"string"}
EMAIL = '' # @param {type:"string"}
REPORT_TITLE = "BOOKSTALL SALES REPORT {time_range}"


In [337]:
# @title HTML Templates

TABLE_CAPTION_TEMPLATE = '<caption>{caption}</caption>'
TABLE_TEMPLATE = '<table>{table_caption}{table_head}{table_body}</table>'
HTML_DOC_TEMPLATE = '''
          <!DOCTYPE html>
          <html lang="en">
          <head>
              <meta charset="UTF-8">
              <meta name="viewport" content="width=device-width, initial-scale=1.0">
              <title>{report_tab_title}</title>
              <style>
                  body {{
                      font-family: Arial, sans-serif;
                      margin: 0;
                      padding: 0;
                  }}

                  .header {{
                      padding: 20px;
                      text-align: center;
                  }}

                  .logo {{
                      display: inline-block;
                      vertical-align: middle;
                  }}

                  .company-info {{
                      display: inline-block;
                      vertical-align: middle;
                      margin-left: 20px;
                  }}

                  .company-name {{
                      font-size: 24px;
                      font-weight: bold;
                      margin-bottom: 5px;
                  }}

                  .company-address {{
                      font-size: 16px;
                  }}

                  .report-content {{
                      padding: 20px;
                      text-align: center;
                  }}

                  table {{
                      margin: auto;
                      border-collapse: collapse;
                      width: 80%;
                      margin-bottom: 20px;
                  }}

                  th, td {{
                      border: 1px solid black;
                      padding: 8px;
                  }}

                  caption {{
                      font-size: 19px;
                      font-weight: bold;
                      caption-side: top;
                      margin-bottom: 10px;
                  }}

                  h1 {{
                      margin-bottom: 30px;
                  }}
              </style>
          </head>
          <body>
              <div class="header">
                  <img src="{logo_url}" alt="Logo" class="logo" width="125" height="125">
                  <div class="company-info">
                      <div class="company-name">{org_name}</div>
                      <div class="company-address">
                          <div>{address_line1}</div>
                          <div>{address_line2}</div>
                          <div>{phone}</div>
                          <div>{email}g</div>
                      </div>
                  </div>
              </div>
              <div class="report-content">
                  <h1>{report_title}</h1>
                  {tables}
              </div>
          </body>
          </html>
        '''

# Code

In [338]:
# @title HTML

def convertDataFrameToHtmlTable(df, keep_index=True, caption=''):
  output = ''
  if df is None or df.empty:
    return output
  html_string = df.to_html(index=keep_index, justify='center')
  soup = BeautifulSoup(html_string, 'html.parser')
  table = soup.find('table')
  if not table:
    return output
  thead = soup.find('thead')
  tbody = soup.find('tbody')
  if thead and tbody:
    output = TABLE_TEMPLATE.format(table_caption=TABLE_CAPTION_TEMPLATE.format(caption=caption),
                                   table_head=thead,
                                   table_body=tbody)
  return output

def getHtmlReport(df_htmls,report_tab_title,logo_url,org_name,address_line1,
                  address_line2,phone,email,report_title):
  tables = ''.join(df_htmls)
  return HTML_DOC_TEMPLATE.format(report_tab_title=report_tab_title,
                                  logo_url=logo_url, org_name=org_name,
                                  address_line1=address_line1,address_line2=address_line2,
                                  phone=phone,email=email,report_title=report_title,
                                  tables=tables)

In [339]:
# @title Utils

def formatTimestamp(timestamp):
  timestamp_datetime = datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%S%z')
  ist_datetime = timestamp_datetime.astimezone(IST_TIMEZONE)
  return ist_datetime.strftime("%d-%m-%Y %H:%M")

def calculate_tax_inclusive_amount(item):
  discount = float(item['discount'].strip('%'))/100 if item['discount'] else 0.0
  return item['rate']*item['quantity']*(1-discount)

def getItemType(item):
  return "Book" if item['hsn_or_sac'] == '49011010' else "Article"

def addCaption(df, caption):
  df.columns = pd.MultiIndex.from_product([[caption], df.columns])
  return df

def formatDateForReport(date):
  return datetime.strptime(date, DATE_FORMAT).strftime('%d-%m-%Y')

def set_borders_and_save(filename):
  thin = Side(border_style="thin", color="000000")
  wb = openpyxl.load_workbook(filename)
  for sheet in wb.sheetnames:
    ws = wb[sheet]
    for row in ws[ws.dimensions]:
      for cell in row:
        cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
  wb.save(filename)

In [340]:
# @title OAuth

def loadOauthToken():
  global AUTH_TOKEN
  print("Select Authorization Token file:")
  uploaded_auth_token = files.upload()
  AUTH_TOKEN = next(iter(uploaded_auth_token.values())).decode()

def renewOauthToken(refresh_token, client_id, client_secret):
  global API_CALLS_COUNTER
  params = []
  params.append("refresh_token={REFRESH_TOKEN}".format(REFRESH_TOKEN=refresh_token))
  params.append("client_id={CLIENT_ID}".format(CLIENT_ID=client_id))
  params.append("client_secret={CLIENT_SECRET}".format(CLIENT_SECRET=client_secret))
  params.append("redirect_uri={REDIRECT_URI}".format(REDIRECT_URI=OAUTH_REDIRECT_URI))
  params.append("grant_type=refresh_token")
  url = OAUTH_ROOT_URL+"&".join(params)
  payload = {}
  headers = {}
  response = requests.request("POST", url, headers=headers, data=payload)
  API_CALLS_COUNTER += 1
  return response.json()

def createSession(access_token):
  session = requests.Session()
  session.headers.update({'Authorization': 'Zoho-oauthtoken {access_token}'.format(access_token=access_token)})
  return session

In [341]:
# @title Organization

def getOrganizationId(session):
  '''
    Only takes the first organization from the list of orgs.
  '''
  global API_CALLS_COUNTER, ORGANIZATION_ID, ORGANIZATION_NAME
  url = ROOT_API_ENDPOINT+'organizations'
  payload = {}
  response = session.get(url,data=payload)
  API_CALLS_COUNTER += 1
  org_info = response.json()
  org = org_info['organizations'][0]
  print("Using Organization: {name}".format(name=org['name']))
  ORGANIZATION_ID = org['organization_id']
  if not ORGANIZATION_NAME:
    ORGANIZATION_NAME = org['name']

In [342]:
# @title Invoices

def getInvoices(session, from_date=None, to_date=None, additional_params=[]):
  global API_CALLS_COUNTER
  params = additional_params
  params.append("organization_id={ORGANIZATION_ID}".format(ORGANIZATION_ID=ORGANIZATION_ID))
  params.append("per_page={MAX_RECORDS_PER_PAGE}".format(MAX_RECORDS_PER_PAGE=MAX_RECORDS_PER_PAGE))
  params.append("sort_column=created_time")
  if from_date is not None:
    params.append("date_start={FROM_DATE}".format(FROM_DATE=from_date))
  if to_date is not None:
    params.append("date_end={TO_DATE}".format(TO_DATE=to_date))
  url = ROOT_API_ENDPOINT+"invoices?"+"&".join(params)

  results = []
  payload = {}
  send_request = True
  current_page = 1
  while send_request:
    response = session.get(url+"&page={PAGE}".format(PAGE=current_page),data=payload)
    API_CALLS_COUNTER += 1
    #perform exception handling here
    results += response.json()['invoices']
    send_request = response.json()['page_context']['has_more_page']
    current_page += 1

  return pd.DataFrame.from_records(results) if results else None

def getInvoiceDetails(session, invoice_id):
  global API_CALLS_COUNTER
  params = []
  params.append("organization_id={ORGANIZATION_ID}".format(ORGANIZATION_ID=ORGANIZATION_ID))
  url = ROOT_API_ENDPOINT+"invoices/{invoice_id}?".format(invoice_id=invoice_id)+"&".join(params)
  payload = {}
  response = session.get(url, data=payload)
  API_CALLS_COUNTER += 1
  return response.json()
  #perform exception handling here

def getPaymentsForInvoice(session, invoice_id):
  global API_CALLS_COUNTER
  params = []
  params.append("organization_id={ORGANIZATION_ID}".format(ORGANIZATION_ID=ORGANIZATION_ID))
  url = ROOT_API_ENDPOINT+"invoices/{invoice_id}/payments?".format(invoice_id=invoice_id)+"&".join(params)
  payload = {}
  response = session.get(url, data=payload)
  API_CALLS_COUNTER += 1
  return response.json()
  #perform exception handling here

In [343]:
# @title Payments

def getPayments(session, from_date=None, to_date=None):
  global API_CALLS_COUNTER
  params = []
  params.append("organization_id={ORGANIZATION_ID}".format(ORGANIZATION_ID=ORGANIZATION_ID))
  params.append("per_page={MAX_RECORDS_PER_PAGE}".format(MAX_RECORDS_PER_PAGE=MAX_RECORDS_PER_PAGE))
  params.append("sort_column=created_time")
  if from_date is not None:
    params.append("date_start={FROM_DATE}".format(FROM_DATE=from_date))
  if to_date is not None:
    params.append("date_end={TO_DATE}".format(TO_DATE=to_date))
  url = ROOT_API_ENDPOINT+"customerpayments?"+"&".join(params)

  results = []
  payload = {}
  send_request = True
  current_page = 1
  while send_request:
    response = session.get(url+"&page={PAGE}".format(PAGE=current_page), data=payload)
    API_CALLS_COUNTER += 1
    #perform exception handling here
    results += response.json()['customerpayments']
    send_request = response.json()['page_context']['has_more_page']
    current_page += 1

  return pd.DataFrame.from_records(results) if results else None

In [344]:
# @title Report Creation

def createDataForReporting(session, from_date=None, to_date=None):
  invoices = getInvoices(session, from_date, to_date)
  if invoices is None:
    return None, None, None, None

  invoice_data_rows = []
  sales_type_data_rows = []
  intra_state_tax_data_rows = []
  for invoice_id in invoices['invoice_id']:
    invoice = getInvoiceDetails(session, invoice_id)['invoice']
    payments = getPaymentsForInvoice(session, invoice_id)['payments']
    #assert len(payments) <= 1
    payment_mode = payments[0]['payment_mode'] if payments else PAYMENT_MODE_CREDIT
    invoice_data_rows.append({'Invoice No.' : invoice['invoice_number'],
                              'Time' : formatTimestamp(invoice['created_time']),
                              'Payment Mode' : payment_mode,
                              'Amount (including tax)' : invoice['bcy_total']})
    for item in invoice['line_items']:
      item_total_inclusive_of_tax = calculate_tax_inclusive_amount(item)
      sales_type_data_rows.append({'Item Type' : getItemType(item),
                                   'Amount (including GST)' : item_total_inclusive_of_tax,
                                   'Payment Mode' : payment_mode})
      cgst_amount, cgst_percent, sgst_amount, sgst_percent = 0.0, '0%', 0.0, '0%'
      for tax in item['line_item_taxes']:
        if tax['tax_name'].startswith("CGST"):
          cgst_amount, cgst_percent = tax['tax_amount'], re.search('\d*%', tax['tax_name']).group()
        elif tax['tax_name'].startswith("SGST"):
          sgst_amount, sgst_percent = tax['tax_amount'], re.search('\d*%', tax['tax_name']).group()
      intra_state_tax_data_rows.append({'Date' : invoice['date'],
                                        'SKU' : item['sku'],
                                        'Amount (excluding GST)' : item['item_total'],
                                        'CGST Amount' : cgst_amount,
                                        'SGST Amount' : sgst_amount,
                                        'CGST Percent' : cgst_percent,
                                        'SGST Percent' : sgst_percent,
                                        'Amount (including GST)' : item_total_inclusive_of_tax})

  invoice_df = pd.DataFrame(data = invoice_data_rows)
  sales_type_df = pd.DataFrame(data = sales_type_data_rows).groupby(['Item Type', 'Payment Mode']).sum()
  intra_state_tax_df = pd.DataFrame(data = intra_state_tax_data_rows)
  total_intra_state_tax_df = intra_state_tax_df[['Amount (excluding GST)','CGST Amount','SGST Amount','Amount (including GST)']].sum().to_frame(name="Total")
  return invoice_df, sales_type_df, intra_state_tax_df, total_intra_state_tax_df

def generateReport(session, from_date=None, to_date=None):
  report_data = createDataForReporting(session, from_date, to_date)
  time_range = FOR_DATE.format(for_date=formatDateForReport(from_date)) if from_date == to_date else FROM_TO_DATE.format(from_date=formatDateForReport(from_date), to_date=formatDateForReport(to_date))

  if all(df is None for df in report_data):
    return (NO_DATA_MESSAGE.format(time_range=time_range),)*len(report_data)

  invoice_df, sales_type_df, intra_state_tax_df, total_intra_state_tax_df = report_data

  invoice_caption = DAILY_SALES_REPORT_CAPTION.format(time_range=time_range)
  sales_type_caption = SALES_TYPE_REPORT_CAPTION.format(time_range=time_range)
  intra_state_tax_caption = INTRA_STATE_TAX_REPORT_CAPTION.format(time_range=time_range)
  total_intra_state_tax_caption = TOTAL_INTRA_STATE_TAX_REPORT_CAPTION.format(time_range=time_range)

  invoice_html = convertDataFrameToHtmlTable(invoice_df, keep_index=False, caption=invoice_caption)
  sales_type_html = convertDataFrameToHtmlTable(sales_type_df, keep_index=True, caption=sales_type_caption)
  intra_state_tax_html = convertDataFrameToHtmlTable(intra_state_tax_df, keep_index=False, caption=intra_state_tax_caption)
  total_intra_state_tax_html = convertDataFrameToHtmlTable(total_intra_state_tax_df, keep_index=True, caption=total_intra_state_tax_caption)

  invoice_df = addCaption(invoice_df, invoice_caption)
  sales_type_df = addCaption(sales_type_df, sales_type_caption)
  intra_state_tax_df = addCaption(intra_state_tax_df, intra_state_tax_caption)
  total_intra_state_tax_df = addCaption(total_intra_state_tax_df, total_intra_state_tax_caption)

  report = getHtmlReport(df_htmls=[invoice_html,sales_type_html,intra_state_tax_html,total_intra_state_tax_html],
                         report_tab_title=REPORT_TAB_TITLE,
                         logo_url=LOGO_URL,
                         org_name=ORGANIZATION_NAME,
                         address_line1=ADDRESS_LINE1,
                         address_line2=ADDRESS_LINE2,
                         phone=PHONE, email=EMAIL,
                         report_title=REPORT_TITLE.format(time_range=time_range))

  excel_filename = FILENAME.format(from_date=from_date, to_date=to_date, extension='xlsx')
  html_filename = FILENAME.format(from_date=from_date, to_date=to_date, extension='html')

  with pd.ExcelWriter(excel_filename) as writer:
    invoice_df.to_excel(writer, sheet_name=SALES_SHEET_NAME, index=True)
    sales_type_df.to_excel(writer, sheet_name=SALES_TYPE_SHEET_NAME, index=True)
    intra_state_tax_df.to_excel(writer, sheet_name=GST_SHEET_NAME, index=True)
    total_intra_state_tax_df.to_excel(writer, sheet_name=TOTAL_GST_SHEET_NAME, index=True)

  set_borders_and_save(excel_filename)
  if DOWNLOAD_EXCEL_REPORT:
    files.download(excel_filename)


  with open(html_filename, "w") as html_file:
    html_file.write(report)

  if DOWNLOAD_HTML_REPORT:
    files.download(html_filename)

  return report, invoice_df, sales_type_df, intra_state_tax_df, total_intra_state_tax_df

# Daily Sales Summary Report Generation

**Policies**

 - For a given Invoice, there can be only one payment mode. For example, an invoice cannot have partial payments via Card(s) and Cash / UPI.
 - All Credit settlement is done fully and at once. No installments. As in the above case, there can not be partial credit and partial payment for an Invoice.

In [ ]:
# @title Initialization

if not AUTH_TOKEN:
  loadOauthToken()

oauth_token = renewOauthToken(AUTH_TOKEN,CLIENT_ID, CLIENT_SECRET)
session = createSession(oauth_token['access_token'])

if not ORGANIZATION_ID:
  getOrganizationId(session)

In [346]:
# @title Report

sales_report_html, invoice_df, sales_type_df, intra_state_tax_df, total_intra_state_tax_df = generateReport(session=session, from_date=FROM_DATE, to_date=TO_DATE)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
display(IPython.display.HTML(sales_report_html))

In [ ]:
invoice_df

In [ ]:
sales_type_df

In [ ]:
intra_state_tax_df

In [ ]:
total_intra_state_tax_df

# API Calls

In [ ]:
print("API calls made = {count}".format(count=API_CALLS_COUNTER))